### Prueba 2

In [ ]:
import os
import wikipedia
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
import warnings

MI_URL_BASE = "https://models.inference.ai.azure.com"
MI_TOKEN = "xxxxxxxxxxxxxxxxxxxxxxx"
MI_MODELO = "gpt-4o"


os.environ["OPENAI_API_BASE"] = MI_URL_BASE
os.environ["OPENAI_API_KEY"] = MI_TOKEN
os.environ["OPENAI_MODEL_NAME"] = MI_MODELO

print(f"Variables de entorno configuradas para el modelo: {MI_MODELO}")

class WikipediaSearchTool(BaseTool):
    name: str = "BuscadorWikipedia"
    description: str = "Busca en Wikipedia un tema y devuelve un resumen detallado. Útil para obtener información sobre personas, lugares o conceptos."
   
    def _run(self, query: str) -> str:
        """Ejecuta la búsqueda en Wikipedia"""
        try:
            wikipedia.set_lang("es")
            summary = wikipedia.summary(query, sentences=5)
            return summary
        except wikipedia.exceptions.PageError:
            return f"No se encontró ninguna página para '{query}'."
        except wikipedia.exceptions.DisambiguationError as e:
            return f"Búsqueda ambigua para '{query}'. Opciones: {e.options[:3]}"
        except Exception as e:
            return f"Error al buscar en Wikipedia: {str(e)}"


# Instanciar la herramienta
wikipedia_tool = WikipediaSearchTool()
print(f"Herramientas cargadas: {[wikipedia_tool.name]}")

#Investigador
investigador = Agent(
    role="Investigador",
    goal="Encontrar y recopilar la información más relevante y precisa sobre un tema solicitado, utilizando las herramientas disponibles.",
    backstory="Eres un experto en buscar datos. Tu especialidad es filtrar el ruido y encontrar la información clave que servirá como base para un reporte.",
    tools=[wikipedia_tool],
    verbose=True,
    allow_delegation=False
)

#Redactor
redactor = Agent(
    role="Redactor Profesional",
    goal="Escribir un informe claro, conciso y bien estructurado en formato markdown, basado en la información proporcionada por el investigador.",
    backstory="Eres un escritor experto. Tomas datos en bruto y los transformas en un reporte pulido y fácil de entender. Tu objetivo es la claridad y la precisión.",
    verbose=True,
    allow_delegation=False
)

print("Agentes definidos.")

def crear_tareas(tema_investigacion):
    #Investigar
    research_task = Task(
        description=f"Investiga a fondo el siguiente tema: '{tema_investigacion}'. Encuentra los puntos clave, fechas importantes y hechos relevantes.",
        expected_output="Un documento con los puntos clave (bullet points) de la información encontrada sobre el tema.",
        agent=investigador
    )


    #Redactar
    write_task = Task(
        description="Toma la investigación proporcionada y redacta un informe completo en formato markdown. El informe debe tener un título, una introducción breve y luego desarrollar los puntos clave.",
        expected_output="Un informe final en formato markdown.",
        agent=redactor,
        context=[research_task]
    )
   
    return [research_task, write_task]

print("\n ¡Hola! Soy un asistente de Investigación y Redacción.")
print("   Escribe 'salir' para terminar.")


while True:
    try:
       
        tema = input("\n ¿Sobre qué tema te gustaría un informe?: ")
        if tema.lower() in ["salir", "exit", "quit"]:
            print("Hasta la Proxima")
            break
       
        # Asignar las tareas
        tasks = crear_tareas(tema)
       
        # Crear el Crew
        crew = Crew(
            agents=[investigador, redactor],
            tasks=tasks,
            process=Process.sequential,
            verbose=True
        )
       
        # Ejecutar el crew
        print(f"\n Iniciando Investigacion para: '{tema}'...")
        result = crew.kickoff()
       
        print("\n" + "="*80)
        print("Generacion de Informe final")
        print("="*80)
        print(result)


    except Exception as e:
        print(f"Ocurrió un error en el agente: {e}")
    except KeyboardInterrupt:
        print("\n¡Hasta la próxima!")
        break



Variables de entorno configuradas para el modelo: gpt-4o
Herramientas cargadas: ['BuscadorWikipedia']
Agentes definidos.

 ¡Hola! Soy un asistente de Investigación y Redacción.
   Escribe 'salir' para terminar.

 Iniciando Investigacion para: 'rocket'...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: be2152e7-0006-4aa6-ad24-6dc7f84c13b5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investigador                                                                                            │
│                                                                                                                 │
│  Task: Investiga a fondo el siguiente tema: 'rocket'. Encuentra los puntos clave, fechas importantes y hechos   │
│  relevantes.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\marti\Ingenier-a-de-Soluciones-con-Inteligencia-Artificial\.venv\Lib\site-packages\rich\live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investigador                                                                                            │
│                                                                                                                 │
│  Thought: Thought: Necesito buscar información relevante y detallada sobre el tema 'rocket' para recopilar los  │
│  puntos clave, incluyendo fechas importantes y hechos destacados.                                               │
│                                                                                                                 │
│  Using Tool: BuscadorWikipedia                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Redactor Profesional                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Informe sobre Tecnología de Cohetes                                                                          │
│                                                                                                                 │
│  ## Introducción                                                                                                │
│                                                                                                                 │
│  Los cohetes han sido una pieza fundamental en el desarrollo de la exploración espacial y tecnología militar    │
│  desde su invención. Su evolución ha estado marcada por avances significativos en la utilización de diferentes  │
│  sistemas de propulsión, desde los clásicos cohetes de combustible sólido hasta propuestas más avanzadas como   │
│  la propulsión termonuclear. Este informe detalla las características principales de estos sistemas y su        │
│  relevancia histórica y tecnológica.                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Cohetes de Combustible Sólido                                                                               │
│                                                                                                                 │
│  Los cohetes de combustible sólido constituyen una de las formas más antiguas de propulsión. Su funcionamiento  │
│  se basa en la utilización de un propulsor sólido compuesto de un reductor y un oxidante. Los principales       │
│  puntos destacados sobre estos cohetes son:                                                                     │
│                                                                                                                 │
│  - **Historia y desarrollo inicial**:                                                                           │
│    - Fueron los primeros cohetes conocidos, utilizados ya en el siglo XIII por los chinos, quienes emplearon    │
│  pólvora como combustible.                                                                                      │
│  - **Ventajas técnicas**:                                                                                       │
│    - Son valorados por su simplicidad de diseño, alta fiabilidad y capacidad de almacenamiento a largo plazo    │
│  del combustible.                                                                                               │
│  - **Uso moderno**:                                                                                             │
│    - En la actualidad se utilizan ampliamente como cohetes aceleradores en misiones espaciales y aplicaciones   │
│  militares debido a su efectividad y fiabilidad.                                                                │
│                                                                                                                 │
│  A pesar de que el desarrollo de cohetes líquidos e híb

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ebfb7a1-cceb-4018-a195-15baca5452b2                                                                     │
│  Agent: Redactor Profesional                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Generacion de Informe final


```markdown
# Informe sobre Tecnología de Cohetes

## Introducción

Los cohetes han sido una pieza fundamental en el desarrollo de la exploración espacial y tecnología militar desde su invención. Su evolución ha estado marcada por avances significativos en la utilización de diferentes sistemas de propulsión, desde los clásicos cohetes de combustible sólido hasta propuestas más avanzadas como la propulsión termonuclear. Este informe detalla las características principales de estos sistemas y su relevancia histórica y tecnológica.

---

## Cohetes de Combustible Sólido

Los cohetes de combustible sólido constituyen una de las formas más antiguas de propulsión. Su funcionamiento se basa en la utilización de un propulsor sólido compuesto de un reductor y un oxidante. Los principales puntos destacados sobre estos cohetes son:

- **Historia y desarrollo inicial**: 
  - Fueron los primeros cohetes conocidos, utilizados ya en el siglo XIII por los chinos, quienes emplearon pólvora como combust

: 